In [1]:
import pandas as pd
import tweepy
import requests
import datetime
import json
import matplotlib.pyplot as plt
from Classes import Data_crawl
from functions import *

### My Access Keys for Tweepy

In [2]:
api_key = 'I3N75MY5qJxCvSrLWe0ZG4vQk'
api_key_secret = 'Q3GC8Vr6F2FNqwh7v8rWoDNnA1Qqnn4RjkJ6STVtAQLamw4qCs'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAIPvbAEAAAAApt3kcfjyiVBOLVWMbi1Fu6CLL24%3DuIMwpFYplQqMd68VXXv4eU954tnOQu5vljRnV3xL292k4F5sFR'
access_token = '1467171706754453504-EJx2MzY1RtEGHXf3hqBZkANnsc4hA2'
access_token_secret = 'VpVdfP1WhID2nas76BXRTMdXXOTvx1BPO1J9LDYs9QM65'

In [3]:
token = list()
with open('tokens.txt', 'r') as f:
    for line in f:
        line = line.replace('\n', '')
        token.append(line)

## <font color="pink">Biden tweets crawling </font> ##



In [10]:
download = False # 需不需要抓資料，FALSE 的話載入本地資料

today  = '2022-11-07' # date you wanna captured
file_path = 'Biden/' + today + '/'
dataset = Data_crawl(token=token, file_path=file_path)
query = 'from:POTUS -is:retweet' 
tweet_fields = ['author_id', 'created_at', 'retweeted', 'retweet_count', 'favorite_count']


if download:
    tweet_df = dataset.Get_Tweets_Dataframe(
        query=query,
        tweet_fields=tweet_fields
    )
else:
    tweet_df = pd.read_excel(file_path + 'Biden.xlsx')
    dataset.Get_Tweets_Dataframe(
        tweet_df=tweet_df, 
        wanna_download=download, 
        query=query, 
        tweet_fields=tweet_fields)
    tweet_df.tail(n=8)
dataset.tweets_df

,id,created_at,text,retweeted,retweet_count,favorite_count
0,1589407281187274800,Mon Nov 07 00:00:00 +0000 2022,Ted Cruz said that folks who are receiving stu...,False,9718,56394
1,1589377088162865200,Sun Nov 06 22:00:01 +0000 2022,I'm doing everything I can so that you and you...,False,2257,6913
2,1589369533072015400,Sun Nov 06 21:30:00 +0000 2022,"Folks, there’s nothing that will create more c...",False,6141,18260
3,1589360169602916400,Sun Nov 06 20:52:47 +0000 2022,This is about giving people a fighting chance ...,False,1650,5706
4,1589331784579301400,Sun Nov 06 19:00:00 +0000 2022,"Together, we've added a record 700,000 manufac...",False,4346,16584
...,...,...,...,...,...,...
995,1544683264601882600,Wed Jul 06 14:02:43 +0000 2022,We can deal with the immediate crisis of high ...,False,5822,41464
996,1544481063027855400,Wed Jul 06 00:39:15 +0000 2022,More than 50 years have passed since the jungl...,False,1733,10697
997,1544468827181817900,Tue Jul 05 23:50:37 +0000 2022,"Today, I stood humbly alongside Specialists Fi...",False,1977,14436
998,1544425047066017800,Tue Jul 05 20:56:39 +0000 2022,https://t.co/BdllhUqViK,False,570,2666


## <font color="pink">依照 tweet ID 尋找每篇推文的 retweeters</font> 

> * 使用 JSON/XLSX 格式儲存 retweeters
> * 針對每篇 tweet 建立兩個 list，分別把抓到的 ID、UserName 存進 Retweeter_List
> * 將抓取的時間範圍寫在最前面
>
> * <font color=#FF6600> 要用 paginator 抓 retweeters，否則會抓不超過 100 個
> * 練習檔案讀寫務必記得拿 White House，否則複寫以後檔案會消失 (2022/10/06 )  </font>


In [ ]:
retweeter_dict = dict() # keys:tweet id, value:list of retweeters
if download:
    retweeter_dict = dataset.Get_Retweeters_Multi()

## <font color="pink"> 分布圖 & Target User </font> 

- 依照每篇 tweet 看一下 retweet count ，畫成 bar chart.

- 依照每個 user 轉推的篇數畫出 bar chart，<font color=#FFD382> X軸為 轉推1,2,3... 的篇數，Y軸為人數 </font>
- 分布圖出來以後，設一個底線，把轉推篇數低於這個底線的 user 踢掉，留下的就是 target user

In [ ]:
Need_Union = False # if u need to unions the files, set as True

if Need_Union:
    retweet_file_list = list() # storing all files name u want to union
    for i in range(len(token)):
        filename = file_path + str(i) + '_retweeters.json'
        retweet_file_list.append(filename)
    retweeter_dict = union_JSON_files(file_list = retweet_file_list)

else: # u wanna load the retweeter json file that has already unioned.
    with open(file_path + today + '_retweeters.json', 'r') as f:
        json_file = json.load(f)
        for tweet_id in json_file:
            retweeter_dict[tweet_id] = json_file[tweet_id]

print("Total tweets :", len(retweeter_dict.keys()))

In [ ]:
# retweet count for each tweet -- bar chart
count = list()
for id in retweeter_dict.keys():
    count.append(len(retweeter_dict[id]))
retweet_count = list(map(str, count))

plt.figure(figsize = (20,5))
plt.title('Biden dataset retweeter count')
plt.bar(retweet_count, count, color=['black', 'blue'], width=0.8, align='center')
plt.grid(True,color='r',alpha=0.6)
plt.xlabel('Retweet count for each tweet')
plt.ylabel('Retweeter count')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# Union the retweeter lists, filtering the repeated id

retweeters = set()
requests_count = 0
for i in retweeter_dict:
    retweeter_list = set(retweeter_dict[i])
    requests_count += len(retweeter_list)
    retweeters |= retweeter_list # Union into retweeters

# And also, see the ratio of retweeters and requests count --> bar chart
plt.figure(figsize = (5,8))
plt.title('Retweeters and Requests count')
plt.bar([str(len(retweeters)), str(requests_count)], [len(retweeters), requests_count],color=['red', 'blue'], width=0.3, align='center')
plt.grid(True,color='r',alpha=0.6)
plt.show()

## Target User
- 踢掉 retweet 篇數過少的 users，剩下的對象就是 target users.
- 接著再用 target users 去蒐集相對應的 followers.
- Negative Sampling 可能要上.

In [ ]:
# start finding retweets count of each user
user_retweet_count = dict()
result = UserRetweetCount_MultiCore(
    retweeter_dict=retweeter_dict, 
    retweeters=list(retweeters), 
    parallelism=8, 
    requests_count=requests_count
)
# parralle degree = 15, 2m19.3s
# parralle degree = 9, 2m43.4s

In [ ]:
# start statistics retweet count

"""
    Retweet count from 1 to len(how many tweets)
    show_list : first element represent 'how many users retweet 1 tweet', so are the rests.

"""
label = [str(i) for i in range(1,58,1)]
show_list = [0 for i in range(tweet_df.shape[0])]

for user in result.keys():
    index = result[user]
    show_list[index-1] += 1

In [ ]:
# Draw a bar plot to see
print("Total users: %d" % len(retweeters))
color = (0.3, 0.9, 0.4, 0.6)
plt.figure(figsize = (35,8))
plt.title("User's retweet count")
plt.bar(label, show_list, color=color, width=0.8, align='center')
plt.grid(True,color='r',alpha=0.3)
plt.xlabel('Retweet count')
plt.ylabel('User Count')
plt.xticks(rotation='vertical')
for index, value in enumerate(show_list):
    plt.text(index, value, str(value))
plt.show()

In [ ]:
#   Draw a bar plot to see after bounded
"""
    Set bound after observing the user's retweet counts distributionns.
    Most of the users distributed in the range of 1 to 3.
"""
bound = 4
print("Total users :", len(retweeters))
print("Deleting users of retweet counts lower than %d : %d" % (bound, sum(show_list[0:bound])))
print("Total of target users : %d" % sum(show_list[bound:]))

color = (0.3, 0.9, 0.4, 0.6)
plt.figure(figsize = (35,8))
plt.title("User's retweet count after bounded")
plt.bar(label[bound:], show_list[bound:], color=color, width=0.8, align='center')
plt.grid(True,color='r',alpha=0.3)
plt.xlabel('Target users retweet count')
plt.ylabel('Target user count')
plt.xticks(rotation='vertical')
for index, value in enumerate(show_list[bound:]):
    plt.text(index, value, str(value))
plt.show()

In [ ]:
wanna_find_target = False
target_users = list()
if wanna_find_target:   # need to run the function to find target users
    retweeters = dict(result)
    target_users = FindTargetUser_MultiCore(
        bound = bound,
        retweeters = retweeters,
    )
    target_users = sorted(target_users)
    print("Target users :", len(target_users))
    with open(file_path + "target_users.json", "w") as f:
        json.dump(target_users, f, indent=True)

else: # load the file of target users
    with open(file_path + "target_users.json", "r") as f:
        json_file = json.load(f)
        target_users = list(json_file)

In [ ]:
# get tweets from the post wall of each target user.
# and dont captured those target users who already been crawed.
"""TargetUser_Tweets = dataset.Get_User_Tweets_Multi(
    file_path=file_path,
    target_user=target_users
)
print(len(TargetUser_Tweets))
"""

'TargetUser_Tweets = dataset.Get_User_Tweets_Multi(\n    file_path=file_path,\n    target_user=target_users\n)\nprint(len(TargetUser_Tweets))\n'

In [15]:
# get follower list of every target users

"""
dataset.Get_Followers_Multi(
    target_users = target_users,
    file_path = file_path
)

## <font color="pink"> Graph Construction </font> ##


1. 把所有 target user 的 id 一起 encoding 成 0, 1, 2, 3, ... 
   
2. 收集每個 target user 的追蹤關係後再下去比對，有出現在清單上面的 user id，就代表他們有 edge
3. 全部用 follow 的關係當 edge 建圖，會有一張 Base Graph
4. 每篇 tweet 以 base graph 為基礎來建 data graph，每一個 data graph 的區別是權重的不同
5. 如果某兩個 user 有共同轉推同一篇 tweet ，那這兩個 user 在這張 data graph 的權重 + 1


In [ ]:
retweeters_set = set() # 把 retweeters 用成集合，方便做 union
requests = int(0) # total capture requests
for tweet_id in json_file:
    retweeters = set(json_file[tweet_id])
    requests += len(retweeters)
    retweeters_set = retweeters_set.union(retweeters) # set union

plt.title('Biden dataset  ' + filename)
plt.grid(True,color='r',alpha=0.6)
v_label = 'Retweeter (' + str(len(retweeters_set)) + ')'
r_label = 'Requests (' + str(requests) + ')'
plt.bar([v_label, r_label], [len(retweeters_set), requests], color=['orange', 'blue'], width=0.4)
plt.figure(figsize = (1,2))
plt.show()

In [ ]:
# 檢查有沒有重複
print(len(retweeters_set.difference()))
retweeters_set = sorted(retweeters_set)

#### 2. Encoding
- 將所有的 retweeter <font color='#FF1212'> (vertex)</font> 的 user id encoding 成 0,1,2,3...，方便建 adjacency matrix
- json file 的 multiple objects 最前面與最後面要加中括弧[]，objects 之間要加逗號

In [ ]:

wanna_save = True
# json file 再多一個 object -> user id : following list
user_follower = dict.fromkeys(retweeters_set)

for id in user_follower.keys():
    followers = []
    for follwer in tweepy.Paginator(client.get_users_followers, id).flatten():
        followers.append(follwer.id)
    user_follower[id] = followers


In [ ]:
if wanna_save:
    with open('2022-10-06_followers.json', 'w') as f: # 這裡記得改掉 不然又寫錯
        foller_json = json.dumps(user_follower)
        f.write(foller_json)

> 接下來要將 user's 的 followers，跟 retweeters_set 做比較，不在 retweeter_set 裡面的 id 直接踢掉
> 
> 剩下來的存起來，用來建 edge list

In [ ]:
encoding_table = dict()
for i,j in zip(retweeters_set, range(len(retweeters_set))):
    encoding_table[i] = j